In [1]:
import numpy as  np
import pandas as pd
import scanpy as sc
import decoupler as dc

# import liana
import liana as li
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean

import sc_atlas_helpers as ah
#from scanpy_helper_submodule import scanpy_helpers as sh

# %%
from tqdm.auto import tqdm
import contextlib
import os
import statsmodels.stats.multitest
import numpy as np
from anndata import AnnData
import scipy.sparse

/home/kvalem/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


# # Functions

In [2]:
def fdr_correction(df, pvalue_col="pvalue", *, key_added="fdr", inplace=False):
    """Adjust p-values in a data frame with test results using FDR correction."""
    if not inplace:
        df = df.copy()

    df[key_added] = statsmodels.stats.multitest.fdrcorrection(df[pvalue_col].values)[1]

    if not inplace:
        return df

In [3]:
# %%
"""Plotting functions for group comparisons"""

import altair as alt
import pandas as pd
import numpy as np


def plot_lm_result_altair(
    df,
    p_cutoff=0.1,
    p_col="fdr",
    x="variable",
    y="group",
    color="coef",
    title="heatmap",
    cluster=False,
    value_max=None,
    configure=lambda x: x.configure_mark(opacity=1),
    cmap="redblue",
    reverse=True,
    domain=lambda x: [-x, x],
    order=None,
):
    """
    Plot a results data frame of a comparison as a heatmap
    """
    df_filtered = df.loc[lambda _: _[p_col] < p_cutoff, :]
    df_subset = df.loc[
        lambda _: _[x].isin(df_filtered[x].unique()) & _[y].isin(df[y].unique())
    ]
    if not df_subset.shape[0]:
        print("No values to plot")
        return

    if order is None:
        order = "ascending"
        if cluster:
            from scipy.cluster.hierarchy import linkage, leaves_list

            values_df = df_subset.pivot(index=y, columns=x, values=color)
            order = values_df.columns.values[
                leaves_list(
                    linkage(values_df.values.T, method="average", metric="euclidean")
                )
            ]

    def _get_significance(fdr):
        if fdr < 0.001:
            return "< 0.001"
        elif fdr < 0.01:
            return "< 0.01"
        elif fdr < 0.1:
            return "< 0.1"
        else:
            return np.nan

    df_subset["FDR"] = pd.Categorical([_get_significance(x) for x in df_subset[p_col]])

    if value_max is None:
        value_max = max(
            abs(np.nanmin(df_subset[color])), abs(np.nanmax(df_subset[color]))
        )
    # just setting the domain in altair will lead to "black" fields. Therefore, we constrain the values themselves.
    df_subset[color] = np.clip(df_subset[color], *domain(value_max))
    return configure(
        alt.Chart(df_subset, title=title)
        .mark_rect()
        .encode(
            x=alt.X(x, sort=order),
            y=y,
            color=alt.Color(
                color,
                scale=alt.Scale(scheme=cmap, reverse=reverse, domain=domain(value_max)),
            ),
        )
        + alt.Chart(df_subset.loc[lambda x: ~x["FDR"].isnull()])
        .mark_point(color="white", filled=True, stroke="black", strokeWidth=0)
        .encode(
            x=alt.X(x, sort=order),
            y=y,
            size=alt.Size(
                "FDR:N",
                scale=alt.Scale(
                    domain=["< 0.001", "< 0.01", "< 0.1"],
                    range=4 ** np.array([3, 2, 1]),
                ),
            ),
        )
    )


In [4]:
# %%
from typing import Sequence, Union
from anndata import AnnData, ImplicitModificationWarning
import numpy as np
import pandas as pd
from operator import and_
from functools import reduce
import warnings


def pseudobulk(
    adata,
    *,
    groupby: Union[str, Sequence[str]],
    aggr_fun=np.sum,
    min_obs=10,
) -> AnnData:
    """
    Calculate Pseudobulk of groups

    Parameters
    ----------
    adata
        annotated data matrix
    groupby
        One or multiple columns to group by
    aggr_fun
        Callback function to calculate pseudobulk. Must be a numpy ufunc supporting
        the `axis` attribute.
    min_obs
        Exclude groups with less than `min_obs` observations

    Returns
    -------
    New anndata object with same vars as input, but reduced number of obs.
    """
    if isinstance(groupby, str):
        groupby = [groupby]

    combinations = adata.obs.loc[:, groupby].drop_duplicates()

    if adata.is_view:
        # for whatever reason, the pseudobulk function is terribly slow when operating on a view.
        adata = adata.copy()

    # precompute masks
    masks = {}
    for col in groupby:
        masks[col] = {}
        for val in combinations[col].unique():
            masks[col][val] = adata.obs[col] == val

    expr_agg = []
    obs = []

    for comb in combinations.itertuples(index=False):
        mask = reduce(and_, (masks[col][val] for col, val in zip(groupby, comb)))
        if np.sum(mask) < min_obs:
            continue
        expr_row = aggr_fun(adata.X[mask, :], axis=0)
        obs_row = comb._asdict()
        obs_row["n_obs"] = np.sum(mask)
        # convert matrix to array if required (happens when aggregating spares matrix)
        try:
            expr_row = expr_row.A1
        except AttributeError:
            pass
        obs.append(obs_row)
        expr_agg.append(expr_row)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ImplicitModificationWarning)
        return AnnData(
            X=np.vstack(expr_agg),
            var=adata.var,
            obs=pd.DataFrame.from_records(obs),
        )



In [5]:
# %%
"""Helper functions for cellphonedb analysis

Focuses on differential cellphonedb analysis between conditions.
"""
from typing import List, Literal
import pandas as pd
#from .pseudobulk import pseudobulk
import numpy as np
import scanpy as sc
import altair as alt
#from .compare_groups.pl import plot_lm_result_altair
#from .util import fdr_correction


class CpdbAnalysis:
    def __init__(
        self, cpdb, adata, *, pseudobulk_group_by: List[str], cell_type_column: str
    ):
        """
        Class that handles comparative cellphonedb analysis.

        Parameters
        ----------
        cpdb
            pandas data frame with cellphonedb interactions.
            Required columns: `source_genesymbols`, `target_genesymbol`.
            You can get this from omnipathdb:
            https://omnipathdb.org/interactions/?fields=sources,references&genesymbols=1&databases=CellPhoneDB
        adata
            Anndata object with the target cells. Will use this to derive mean fraction of expressed cells.
            Should contain counts in X.
        pseudobulk_group_by
            See :func:`scanpy_helper.pseudobulk.pseudobulk`. Pseudobulk is used to compute the mean fraction
            of expressed cells by patient
        cell_type_column
            Column in anndata that contains the cell-type annotation.
        """
        self.cpdb = cpdb
        self.cell_type_column = cell_type_column
        self._find_expressed_genes(adata, pseudobulk_group_by)

    def _find_expressed_genes(self, adata, pseudobulk_group_by):
        """Compute the mean expression and fraction of expressed cells per cell-type.
        This is performed on the pseudobulk level, i..e. the mean of means per patient is calculated.
        """
        pb_fracs = pseudobulk(
            adata,
            groupby=pseudobulk_group_by + [self.cell_type_column],
            aggr_fun=lambda x, axis: np.sum(x > 0, axis) / x.shape[axis],  # type: ignore
        )
        fractions_expressed = pseudobulk(
            pb_fracs, groupby=self.cell_type_column, aggr_fun=np.mean
        )
        fractions_expressed.obs.set_index(self.cell_type_column, inplace=True)

        pb = pseudobulk(
            adata,
            groupby=pseudobulk_group_by + [self.cell_type_column],
        )
        sc.pp.normalize_total(pb, target_sum=1e6)
        sc.pp.log1p(pb)
        pb_mean_cell_type = pseudobulk(
            pb, groupby=self.cell_type_column, aggr_fun=np.mean
        )
        pb_mean_cell_type.obs.set_index(self.cell_type_column, inplace=True)

        self.expressed_genes = (
            fractions_expressed.to_df()
            .melt(ignore_index=False, value_name="fraction_expressed")
            .reset_index()
            .merge(
                pb_mean_cell_type.to_df()
                .melt(ignore_index=False, value_name="expr_mean")
                .reset_index(),
                on=[self.cell_type_column, "variable"],
            )
        )

    def significant_interactions(
        self,
        de_res: pd.DataFrame,
        *,
        pvalue_col="pvalue",
        fc_col="log2FoldChange",
        gene_symbol_col="gene_id",
        max_pvalue=0.1,
        min_abs_fc=1,
        adjust_fdr=True,
        min_frac_expressed=0.1,
        de_genes_mode: Literal["ligand", "receptor"] = "ligand",
    ) -> pd.DataFrame:
        """
        Generates a data frame of differentiall cellphonedb interactions.

        This function will extract all known ligands (or receptors, respectively) from a list of differentially expressed
        and find all receptors (or ligands, respectively) that are expressed above a certain cutoff in all cell-types.

        Parameters:
        -----------
        de_res
            List of differentially expressed genes
        pvalue_col
            column in de_res that contains the pvalue or false discovery rate
        gene_id_col
            column in de_res that contains the gene symbol
        min_frac_expressed
            Minimum fraction cells that need to express the receptor (or ligand) to be considered a potential interaction
        de_genes_mode
            If the list of de genes provided are ligands (default) or receptors. In case of `ligand`, cell-types
            that express corresonding receptors above the threshold will be identified. In case of `receptor`,
            cell-types that express corresponding ligands above the threshold will be identified.
        adjust_fdr
            If True, calculate false discovery rate on the pvalue, after filtering for genes that are contained
            in the cellphonedb.
        """
        if de_genes_mode == "ligand":
            cpdb_de_col = "source_genesymbol"
            cpdb_expr_col = "target_genesymbol"
        elif de_genes_mode == "receptor":
            cpdb_de_col = "target_genesymbol"
            cpdb_expr_col = "source_genesymbol"
        else:
            raise ValueError("Invalud value for de_genes_mode!")

        de_res = de_res.loc[lambda x: x[gene_symbol_col].isin(self.cpdb[cpdb_de_col])]
        if adjust_fdr:
            de_res = fdr_correction(de_res, pvalue_col=pvalue_col, key_added="fdr")
            pvalue_col = "fdr"

        significant_genes = de_res.loc[
            lambda x: (x[pvalue_col] < max_pvalue) & (np.abs(x[fc_col]) >= min_abs_fc),
            gene_symbol_col,
        ].unique()  # type: ignore
        significant_interactions = self.cpdb.loc[
            lambda x: x[cpdb_de_col].isin(significant_genes)
        ]

        res_df = (
            self.expressed_genes.loc[
                lambda x: x["fraction_expressed"] >= min_frac_expressed
            ]  # type: ignore
            .merge(
                significant_interactions,
                left_on="variable",
                right_on=cpdb_expr_col,
            )
            .drop(columns=["variable"])
            .merge(de_res, left_on=cpdb_de_col, right_on=gene_symbol_col)
            .drop(columns=[gene_symbol_col])
        )

        return res_df

    def plot_result(
        self,
        cpdb_res,
        *,
        pvalue_col="fdr",
        group_col="group",
        fc_col="log2FoldChange",
        title="CPDB analysis",
        aggregate=True,
        clip_fc_at=(-5, 5),
        label_limit=100,
        cluster: Literal["heatmap", "dotplot"] = "dotplot",
        de_genes_mode: Literal["ligand", "receptor"] = "ligand",
    ):
        """
        Plot cpdb results as heatmap

        Parameters
        ----------
        cpdb_res
            result of `significant_interactions`. May be further filtered or modified.
        group_col
            column to be used for the y axis of the heatmap
        aggregate
            whether to merge multiple targets of the same ligand into a single column
        de_genes_mode
            If the list of de genes provided are ligands (default) or receptors. If receptor, will show the dotplot
            at the top (source are expressed ligands) and the de heatmap at the bottom (target are the DE receptors).
            Otherwise the other way round.
        """
        if de_genes_mode == "ligand":
            cpdb_de_col = "source_genesymbol"
            cpdb_expr_col = "target_genesymbol"
        elif de_genes_mode == "receptor":
            cpdb_de_col = "target_genesymbol"
            cpdb_expr_col = "source_genesymbol"
        else:
            raise ValueError("Invalud value for de_genes_mode!")

        cpdb_res[fc_col] = np.clip(cpdb_res[fc_col], *clip_fc_at)

        # aggregate if there are multiple receptors per ligand
        if aggregate:
            cpdb_res = (
                cpdb_res.groupby(
                    [
                        self.cell_type_column,
                        cpdb_de_col,
                        fc_col,
                        pvalue_col,
                        group_col,
                    ]
                )
                .agg(
                    n=(cpdb_expr_col, len),
                    fraction_expressed=("fraction_expressed", np.max),
                    expr_mean=("expr_mean", np.max),
                )
                .reset_index()
                .merge(
                    cpdb_res.groupby(cpdb_de_col).agg(
                        **{
                            cpdb_expr_col: (
                                cpdb_expr_col,
                                lambda x: "|".join(np.unique(x)),
                            )
                        }
                    ),
                    on=cpdb_de_col,
                )
            )

        cpdb_res["interaction"] = [
            f"{s}_{t}" for s, t in zip(cpdb_res[cpdb_de_col], cpdb_res[cpdb_expr_col])
        ]

        # cluster heatmap
        if cluster is not None:
            from scipy.cluster.hierarchy import linkage, leaves_list

            _idx = self.cell_type_column if cluster == "dotplot" else group_col
            _values = "fraction_expressed" if cluster == "dotplot" else fc_col
            _columns = "interaction"
            values_df = (
                cpdb_res.loc[:, [_idx, _values, _columns]]
                .drop_duplicates()
                .pivot(
                    index=_idx,
                    columns=_columns,
                    values=_values,
                )
                .fillna(0)
            )
            order = values_df.columns.values[
                leaves_list(
                    linkage(values_df.values.T, method="average", metric="euclidean")
                )
            ]
        else:
            order = "ascending"

        p1 = plot_lm_result_altair(
            cpdb_res,
            color=fc_col,
            p_col=pvalue_col,
            x="interaction",
            configure=lambda x: x,
            title="",
            order=order,
            p_cutoff=1,
        ).encode(
            x=alt.X(
                title=None,
                axis=alt.Axis(
                    labelExpr="split(datum.label, '_')[0]",
                    orient="top" if de_genes_mode == "receptor" else "bottom",
                ),
            )
        )

        p2 = (
            alt.Chart(cpdb_res)
            .mark_circle()
            .encode(
                x=alt.X(
                    "interaction",
                    axis=alt.Axis(
                        grid=True,
                        orient="bottom" if de_genes_mode == "receptor" else "top",
                        title=None,
                        labelExpr="split(datum.label, '_')[1]",
                        labelLimit=label_limit,
                    ),
                    sort=order,
                ),
                y=alt.Y(self.cell_type_column, axis=alt.Axis(grid=True), title=None),
                size=alt.Size("fraction_expressed"),
                color=alt.Color("expr_mean", scale=alt.Scale(scheme="cividis")),
            )
        )

        if de_genes_mode == "receptor":
            p1, p2 = p2, p1

        return (
            alt.vconcat(p1, p2, title=title)
            .resolve_scale(size="independent", color="independent", x="independent")
            .configure_mark(opacity=1)
            .configure_concat(spacing=label_limit - 130)
        )



In [16]:
adata = sc.read_h5ad("/data/projects/2022/CRCA/results/v0.1/crc-atlas-dataset/latest/ds_analyses/liana_cell2cell/core_atlas/adata_rank_agregate.h5ad")

In [18]:
d = sc.read_h5ad("/data/projects/2022/CRCA/results/v0.1/crc-atlas-dataset/latest/ds_analyses/liana_cell2cell/core_atlas/tumor_normal/01_subset_adata/adata_tumor_normal_matched.h5ad")

In [21]:
set(d.obs.cell_type)

{'B cell',
 'Cancer cell',
 'DC mature',
 'Endothelial',
 'Enteroendocrine',
 'Epithelial',
 'Fibroblast',
 'Macrophage',
 'Mast cell',
 'Monocyte',
 'NK',
 'Neutrophil',
 'Pericyte',
 'Plasma cell',
 'Schwann cell',
 'T cell CD4',
 'T cell CD8',
 'T cell regulatory',
 'cDC',
 'pDC'}

In [23]:
set(adata.obs.cell_type)

{'B cell',
 'Cancer cell',
 'Endothelial',
 'Fibroblast',
 'Macrophage',
 'Mast cell',
 'Monocyte',
 'NK',
 'Neutrophil',
 'Pericyte',
 'Plasma cell',
 'Schwann cell',
 'T cell CD4',
 'T cell CD8',
 'T cell regulatory',
 'cDC',
 'pDC'}

In [16]:
set(cpdb_sig_int.cell_type)

{'B cell',
 'Cancer cell',
 'Endothelial',
 'Fibroblast',
 'Macrophage',
 'Mast cell',
 'Monocyte',
 'NK',
 'Neutrophil',
 'Pericyte',
 'Plasma cell',
 'Schwann cell',
 'T cell CD4',
 'T cell CD8',
 'T cell regulatory',
 'cDC',
 'pDC'}

In [7]:
modified_list = [item.replace(" ", "_") for item in list(set(adata.obs.cell_type))]

In [8]:
# Folder containing the files
folder_path = "/data/projects/2022/CRCA/results/v0.1/crc-atlas-dataset/latest/ds_analyses/liana_cell2cell/core_atlas/tumor_normal/03_deseq2/"

# Initialize an empty list to hold all DataFrames
df_list = []

# Loop through each element in the modified_list
for element in modified_list:
    # Find the file that starts with the element name
    for file_name in os.listdir(folder_path):
        if file_name.startswith(element):
            # Read the CSV file into a DataFrame
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path,sep="\t")

            # Add a new column "cell_name" with the value of the element
            df["cell_name"] = element

            # Append the DataFrame to the list
            df_list.append(df)
            break  # Break after finding the matching file

# Concatenate all DataFrames into one
final_df = pd.concat(df_list, ignore_index=True)

In [49]:
folder_path = "/data/projects/2022/CRCA/results/v0.1/crc-atlas-dataset/latest/ds_analyses/liana_cell2cell/core_atlas/tumor_normal/03_deseq2/"

from pathlib import Path

de_res = {}
for f in Path(de_res_dir).glob("*DESeq2_result.tsv"):
    ct = f.name.replace("-DESeq2_result.tsv", "").replace("_", " ")
    de_res[ct] = pd.read_csv(f, sep="\t")

# %%
de_res_all2 = pd.concat([df.assign(cell_type=ct) for ct, df in de_res.items()])

In [25]:
de_res_dir="/data/projects/2022/CRCA/results/v1/final/liana_cell2cell/core_atlas/tumor_normal/epithelial_cancer/03_deseq2/tumor_paired/deseq2_dgea"
from pathlib import Path

de_res = {}
for f in Path(de_res_dir).glob("*DESeq2_result.tsv"):
    ct = f.name.replace("-DESeq2_result.tsv", "").replace("_", " ")
    de_res[ct] = pd.read_csv(f, sep="\t")

# %%
de_res_all = pd.concat([df.assign(cell_type=ct) for ct, df in de_res.items()])

In [27]:
set(de_res_all.cell_type)

'B cell memory','B cell'
'Endothelial lymphatic','Endothelial'
 'Fibroblast S1','Fibroblast'
'Plasma IgG', 'Plasma cell'
 'Plasmablast' 'Plasma cell'
 'Treg','T cell regulatory'
 'DC mature', 'cDC'
 'DC3','cDC'
 'cDC1','cDC'
 'CD4 cycling', 'T cell CD4'
 'CD8 cycling','T cell CD8'

{'B cell memory',
 'CD4 cycling',
 'CD8 cycling',
 'DC mature',
 'DC3',
 'Endothelial lymphatic',
 'Enteroendocrine',
 'Epithelial',
 'Fibroblast S1',
 'ILC',
 'NK',
 'Neutrophil',
 'Plasma IgG',
 'Plasmablast',
 'Schwann cell',
 'Treg',
 'cDC1',
 'gamma',
 'pDC'}

In [33]:
# Dictionary to map old cell_type values to new ones
rename_dict = {
    'B cell memory': 'B cell',
    'Endothelial lymphatic': 'Endothelial',
    'Fibroblast S1': 'Fibroblast',
    'Plasma IgG': 'Plasma cell',
    'Plasmablast': 'Plasma cell',
    'Treg': 'T cell regulatory',
    'DC mature': 'cDC',
    'DC3': 'cDC',
    'cDC1': 'cDC',
    'CD4 cycling': 'T cell CD4',
    'CD8 cycling': 'T cell CD8',
    'Enteroendocrine':'Endothelial',
    'ILC':'T cell CD8',
    'gamma':'T cell CD8'
}

# Replace the values in the "cell_type" column based on the dictionary
de_res_all['cell_type'] = de_res_all['cell_type'].replace(rename_dict)

In [41]:
de_res_all = de_res_all.rename(columns={'symbol':"gene_id"})

In [51]:
de_res_all2 = de_res_all2.rename(columns={'symbol':"gene_id"})

In [34]:
set(de_res_all.cell_type)

{'B cell',
 'Endothelial',
 'Epithelial',
 'Fibroblast',
 'NK',
 'Neutrophil',
 'Plasma cell',
 'Schwann cell',
 'T cell CD4',
 'T cell CD8',
 'T cell regulatory',
 'cDC',
 'pDC'}

In [35]:
set(cpdb_sig_int.cell_type)

{'B cell',
 'Cancer cell',
 'Endothelial',
 'Fibroblast',
 'Macrophage',
 'Mast cell',
 'Monocyte',
 'NK',
 'Neutrophil',
 'Pericyte',
 'Plasma cell',
 'Schwann cell',
 'T cell CD4',
 'T cell CD8',
 'T cell regulatory',
 'cDC',
 'pDC'}

In [9]:
de_res = final_df

In [10]:
de_res['cell_name'] = de_res['cell_name'].str.replace('_', ' ')

In [47]:
set(de_res_all.cell_type)

{'B cell',
 'Endothelial',
 'Epithelial',
 'Fibroblast',
 'NK',
 'Neutrophil',
 'Plasma cell',
 'Schwann cell',
 'T cell CD4',
 'T cell CD8',
 'T cell regulatory',
 'cDC',
 'pDC'}

In [48]:
set(adata.obs.cell_type)

{'B cell',
 'Cancer cell',
 'Endothelial',
 'Fibroblast',
 'Macrophage',
 'Mast cell',
 'Monocyte',
 'NK',
 'Neutrophil',
 'Pericyte',
 'Plasma cell',
 'Schwann cell',
 'T cell CD4',
 'T cell CD8',
 'T cell regulatory',
 'cDC',
 'pDC'}

In [52]:

# %%
#result of `significant_interactions`. May be further filtered or modified.
cpdb_res = adata.uns['rank_aggregate'].loc[
        lambda x: x["specificity_rank"] <= 0.01
    ]

# %%
# rename columns in liana results 
cpdb_res=cpdb_res.rename(columns={"ligand_complex":"source_genesymbol","receptor_complex":"target_genesymbol"})

# %%
cpdb_res.columns

# %%
# use scanpy helper class CpdbAnalysis to compute pseudobulk, cell fraction and 
cpdba = CpdbAnalysis(
    cpdb_res,
    adata,
    pseudobulk_group_by=["patient_id"],
    cell_type_column="cell_type"
)

# %%
cpdba

# %%
cpdb_sig_int = cpdba.significant_interactions(
    de_res_all2, max_pvalue=0.1
)

# %%


In [43]:
#columns_to_replace = ['cell_type', 'source', 'target']
#cpdb_sig_int[columns_to_replace] = cpdb_sig_int[columns_to_replace].replace({'cDC': 'Dendritic cell', 'pDC': 'Dendritic cell'})


In [53]:
cpdb_sig_int

,cell_type_x,fraction_expressed,expr_mean,source,target,source_genesymbol,target_genesymbol,lr_means,cellphone_pvals,expr_prod,...,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,weight,comparison,cell_type_y,fdr
0,Cancer cell,0.149646,2.016727,Neutrophil,Schwann cell,CXCL8,SDC3,1.754381,0.0,0.806840,...,69.385540,5.229754,0.562938,9.290112,1.541259e-20,8.716272e-18,1.000000,tumor_vs_normal,Fibroblast S1,3.883302e-19
1,Cancer cell,0.149646,2.016727,Neutrophil,Schwann cell,CXCL8,SDC3,1.754381,0.0,0.806840,...,50.023148,2.409619,1.000612,2.408145,1.603383e-02,9.992474e-01,1.000000,tumor_vs_normal,pDC,3.615409e-02
2,Cancer cell,0.149646,2.016727,Neutrophil,Schwann cell,CXCL8,SDC3,1.754381,0.0,0.806840,...,13.649447,2.251672,0.692260,3.252638,1.143390e-03,1.174123e-02,1.065627,tumor_vs_normal,Schwann cell,3.478185e-03
3,Cancer cell,0.149646,2.016727,Neutrophil,Schwann cell,CXCL8,SDC3,1.754381,0.0,0.806840,...,293.180328,0.349132,0.449523,0.776673,4.373518e-01,9.993719e-01,1.000000,tumor_vs_normal,DC3,5.781782e-01
4,Cancer cell,0.149646,2.016727,Neutrophil,Schwann cell,CXCL8,SDC3,1.754381,0.0,0.806840,...,3.244034,2.225004,0.694730,3.202687,1.361520e-03,1.685540e-02,1.000000,tumor_vs_normal,gamma,4.067015e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17732,Schwann cell,0.110253,2.485961,Pericyte,Schwann cell,COL18A1,GPC4,0.762289,0.0,0.145144,...,13.649193,1.316913,0.198139,6.646419,3.003087e-11,1.300155e-10,0.753878,tumor_vs_normal,Epithelial,3.026589e-10
17733,Schwann cell,0.110253,2.485961,Pericyte,Schwann cell,COL18A1,GPC4,0.762289,0.0,0.145144,...,40.516931,0.858156,0.155373,5.523202,3.328775e-08,8.686158e-07,0.884606,tumor_vs_normal,Endothelial lymphatic,2.366902e-07
17734,Schwann cell,0.110253,2.485961,Pericyte,Schwann cell,COL18A1,GPC4,0.762289,0.0,0.145144,...,3.901726,-0.879704,0.364658,-2.412410,1.584745e-02,6.112068e-02,0.798758,tumor_vs_normal,Plasmablast,3.587344e-02
17735,Schwann cell,0.110253,2.485961,Pericyte,Schwann cell,COL18A1,GPC4,0.762289,0.0,0.145144,...,6.870759,-0.607168,0.356138,-1.704870,8.821873e-02,1.802170e-01,1.091076,tumor_vs_normal,Plasma IgG,1.580301e-01


In [ ]:
import seaborn as sns
sns.histplot(data=cpdb_sig_int, x="log2FoldChange")

In [ ]:
set(cpdb_sig_int.cell_type)

In [17]:
## This is input for CIRCOS PLOT 
cpdb_sig_int.to_csv("epithelial_cancer_old_atlas.csv")

In [17]:
adata.obs.sample_type

Che_2021-SRR14710618-AAACCTGAGCGATAGC     tumor
Che_2021-SRR14710618-AAACCTGAGCGTGTCC     tumor
Che_2021-SRR14710618-AAACCTGCAAGGTGTG     tumor
Che_2021-SRR14710618-AAACCTGCACAGCCCA     tumor
Che_2021-SRR14710618-AAACCTGCACGACTCG     tumor
                                          ...  
Zheng_2022-GSM4904245-TTTGGTTGTCTCTCTG    tumor
Zheng_2022-GSM4904245-TTTGGTTTCCTAGAAC    tumor
Zheng_2022-GSM4904245-TTTGGTTTCGGTTAAC    tumor
Zheng_2022-GSM4904245-TTTGTCAAGTGTTGAA    tumor
Zheng_2022-GSM4904245-TTTGTCAAGTTAGGTA    tumor
Name: sample_type, Length: 541643, dtype: category
Categories (1, object): ['tumor']

pseudobulk and deseq 

In [7]:
pdata = dc.get_pseudobulk(
    adata,
    sample_col='sample_id',
    groups_col=['sample_type',"cell_type"],
    layer='counts',
    mode='sum',
    min_cells=0,
    min_counts=0
)

/home/kvalem/.conda/envs/liana_2024/lib/python3.11/site-packages/decoupler/utils_anndata.py:181: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
/home/kvalem/.local/lib/python3.11/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.


In [10]:
resDir = '/data/projects/2022/CRCA/results/v1/final/liana_cell2cell/core_atlas/'

In [13]:
comparison="tumor_normal"

In [14]:
pdata.var_names.name = "gene_id"

colData = pdata.obs
colData.index.name = "sample_col"

colData.to_csv(f"{resDir}/02_pseudobulk/{comparison}_colData.csv")
rowData = pdata.var[["Geneid", "GeneSymbol", "Chromosome", "Class", "Length"]]
rowData.to_csv(f"{resDir}/02_pseudobulk/{comparison}_rowData.csv")
count_mat = pdata.to_df().T
count_mat.index.name = "gene_id"
count_mat.to_csv(f"{resDir}/02_pseudobulk/{comparison}_count_mat.csv")